# Yellowstone Case Study — Data Lab (Student Starter)

Use this Colab notebook to complete the **Python** portion of the Data Lab.

**You will:**
- Upload your cleaned Excel workbook
- Map the correct column names (so your code works even if the dataset headers differ)
- Create **Tables 1–3** and export them as **CSV + PNG images**
- Create **Figures 1–4** and export them as **PNG images**
- Export a **GIS-ready CSV** (if coordinates exist) for ArcGIS Pro

**What you still do in ArcGIS Pro (outside Colab):**
- Import the GIS-ready CSV (XY points), symbolize, add basemap/park boundary, export Map 1.

> If a cell errors: read the message, fix your mapping or cleaning, and rerun from that section.


In [ ]:

# STEP 0 — Imports and outputs folder
import os, re, zipfile
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import files

OUTDIR = Path("outputs")
OUTDIR.mkdir(exist_ok=True)
print("Outputs will be saved to:", OUTDIR.resolve())


## STEP 1 — Upload your cleaned Excel workbook

Upload the Excel file you created in the **Excel cleaning** part of the Data Lab.
It should contain (or equivalent):
- `Clean Soil Chem (YNP)`
- `Clean Soil Temp (YNP)`
- `Clean Water Chem`

If your sheet names differ, you can still proceed — you’ll edit them in the next step.


In [ ]:

uploaded = files.upload()
excel_path = next(iter(uploaded.keys()))
excel_path


In [ ]:

# Look at the available sheet names
xls = pd.ExcelFile(excel_path)
xls.sheet_names


## STEP 2 — Load the cleaned sheets

**TODO:** If your sheet names differ, edit the three variables below.


In [ ]:

# TODO: edit these names to match *your* workbook
SHEET_SOIL_CHEM  = "Clean Soil Chem (YNP)"
SHEET_SOIL_TEMP  = "Clean Soil Temp (YNP)"
SHEET_WATER_CHEM = "Clean Water Chem"

soil  = pd.read_excel(excel_path, sheet_name=SHEET_SOIL_CHEM)
temp  = pd.read_excel(excel_path, sheet_name=SHEET_SOIL_TEMP)
water = pd.read_excel(excel_path, sheet_name=SHEET_WATER_CHEM)

print("Soil chem:", soil.shape)
print("Soil temp:", temp.shape)
print("Water chem:", water.shape)


## STEP 3 — Inspect columns (then fill in your column mapping)

Run these cells and **copy/paste** column names into the mapping cell.


In [ ]:
print('SOIL CHEM COLUMNS:\n', list(soil.columns))

In [ ]:
print('SOIL TEMP COLUMNS:\n', list(temp.columns))

In [ ]:
print('WATER CHEM COLUMNS:\n', list(water.columns))

### STEP 3A — Column mapping (YOU MUST EDIT)

**TODO:** Replace each `...` with the correct column name string from your data.
- If you have a **Year** column, set `SOIL_YEAR_COL`.
- If you have a **Date** column instead, set `SOIL_DATE_COL` and leave `SOIL_YEAR_COL = None`.

For water chemistry:
- `WATER_SITE_COL` must contain values that include **Bechler River**.
- Pick a short list (3–8) of numeric chemistry columns in `WATER_CHEM_COLS`.


In [ ]:

# -------------------------
# SOIL CHEM (Table 1, Fig 1–2)
# -------------------------
SOIL_SITE_COL = ...      # e.g., "Site"
SOIL_YEAR_COL = ...      # e.g., "Year" (or set to None if you will use a date)
SOIL_DATE_COL = None     # e.g., "SampleDate" (use this if SOIL_YEAR_COL is None)

SOIL_PH_COL = ...        # e.g., "pH"
SOIL_EC_COL = ...        # e.g., "EC"
SOIL_OM_COL = ...        # e.g., "OrganicMatter"
SOIL_BD_COL = ...        # e.g., "BulkDensity"
SOIL_TC_COL = ...        # e.g., "TotalCarbon"

# -------------------------
# SOIL TEMP (Table 2, Fig 3)
# -------------------------
TEMP_SITE_COL  = ...     # e.g., "Site" or "Summit"
TEMP_DATE_COL  = ...     # e.g., "Date"
TEMP_VALUE_COL = ...     # e.g., "SoilTemp_C"
TEMP_PLOT_COL  = ...     # e.g., "Plot" (should include "E22")

# -------------------------
# WATER CHEM (Table 3, Fig 4)
# -------------------------
WATER_SITE_COL = ...     # e.g., "Area" or "SiteName" (must include "Bechler River")
WATER_DATE_COL = None    # optional; set if you want time plots
WATER_DISCHARGE_COL = None  # optional; set if present

# TODO: Choose 3–8 chemistry columns to summarize/plot
WATER_CHEM_COLS = [
    # "Cl_mgL",
    # "SO4_mgL",
    # "Na_mgL",
]


## STEP 4 — Helper functions (provided)

These make your life easier:
- convert text like `<0.2` to numeric
- save a table as a PNG image
- save figures to the outputs folder


In [ ]:

def to_numeric_clean(s: pd.Series) -> pd.Series:
    # Removes common non-numeric symbols before numeric conversion
    if s.dtype == "O":
        s = (s.astype(str)
               .str.replace("<", "", regex=False)
               .str.replace("—", "", regex=False)
               .str.replace("-", "", regex=False)
               .str.strip())
    return pd.to_numeric(s, errors="coerce")

def save_df_as_png(df: pd.DataFrame, path: Path, title: str = None, dpi: int = 300):
    # Saves a dataframe as an image using a matplotlib table
    df_disp = df.copy()
    for c in df_disp.columns:
        if pd.api.types.is_numeric_dtype(df_disp[c]):
            df_disp[c] = df_disp[c].map(lambda x: f"{x:.3g}" if pd.notnull(x) else "")

    fig, ax = plt.subplots(figsize=(min(16, 1 + 0.9*len(df_disp.columns)),
                                    min(10, 1 + 0.35*len(df_disp) + (0.6 if title else 0))))
    ax.axis("off")
    if title:
        ax.set_title(title, pad=12)

    tbl = ax.table(cellText=df_disp.values,
                   colLabels=df_disp.columns.tolist(),
                   loc="center",
                   cellLoc="center")
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9)
    tbl.scale(1, 1.2)

    fig.tight_layout()
    fig.savefig(path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

def save_fig(path: Path, dpi: int = 300):
    plt.savefig(path, dpi=dpi, bbox_inches="tight")


## STEP 5 — Table 1 (Soil chemistry, **2016 only**)

**Deliverables**
- `outputs/Table1_SoilChem_2016.csv`
- `outputs/Table1_SoilChem_2016.png`


In [ ]:

soil2 = soil.copy()

# Create a Year column
if SOIL_YEAR_COL is not None:
    soil2["Year"] = to_numeric_clean(soil2[SOIL_YEAR_COL])
elif SOIL_DATE_COL is not None:
    soil2["Year"] = pd.to_datetime(soil2[SOIL_DATE_COL], errors="coerce").dt.year
else:
    raise ValueError("Set SOIL_YEAR_COL or SOIL_DATE_COL in your mapping.")

# Convert chemistry columns to numeric
for col in [SOIL_PH_COL, SOIL_EC_COL, SOIL_OM_COL, SOIL_BD_COL, SOIL_TC_COL]:
    soil2[col] = to_numeric_clean(soil2[col])

soil2016 = soil2.loc[soil2["Year"] == 2016].copy()
print("Rows in 2016 subset:", len(soil2016))

table1 = (
    soil2016
    .groupby(SOIL_SITE_COL)[[SOIL_PH_COL, SOIL_EC_COL, SOIL_OM_COL, SOIL_BD_COL, SOIL_TC_COL]]
    .agg(["mean", "std", "count"])
)

table1


In [ ]:

# Flatten MultiIndex columns for clean export
table1_flat = table1.copy()
table1_flat.columns = [f"{a}_{b}" for a, b in table1_flat.columns]
table1_flat = table1_flat.reset_index()

table1_csv = OUTDIR / "Table1_SoilChem_2016.csv"
table1_png = OUTDIR / "Table1_SoilChem_2016.png"
table1_flat.to_csv(table1_csv, index=False)
save_df_as_png(table1_flat, table1_png, title="Table 1 — Soil Chemistry Summary (2016)")

print("Saved:", table1_csv.name, "and", table1_png.name)


## STEP 6 — Figures 1–2 (Soil chemistry; grouped bars + SD)

**Figure 1:** pH, EC, Bulk Density
**Figure 2:** Organic Matter, Total Carbon

**Deliverables**
- `outputs/Figure1_SoilChem_BulkParams.png`
- `outputs/Figure2_SoilChem_Carbon.png`


In [ ]:

def plot_grouped_bars(means_df: pd.DataFrame, std_df: pd.DataFrame, title: str, ylabel: str, outpath: Path):
    sites = means_df.index.tolist()
    params = means_df.columns.tolist()

    x = np.arange(len(params))
    width = 0.8 / max(1, len(sites))

    plt.figure(figsize=(10, 5))
    for i, site in enumerate(sites):
        plt.bar(
            x + i*width - (len(sites)-1)*width/2,
            means_df.loc[site].values,
            width=width,
            yerr=std_df.loc[site].values,
            capsize=3,
            label=str(site)
        )
    plt.xticks(x, params)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.legend(title="Site", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    save_fig(outpath)
    plt.close()


In [ ]:

# Figure 1
means1 = soil2016.groupby(SOIL_SITE_COL)[[SOIL_PH_COL, SOIL_EC_COL, SOIL_BD_COL]].mean()
stds1  = soil2016.groupby(SOIL_SITE_COL)[[SOIL_PH_COL, SOIL_EC_COL, SOIL_BD_COL]].std()

means1 = means1.rename(columns={SOIL_PH_COL:"pH", SOIL_EC_COL:"EC", SOIL_BD_COL:"Bulk Density"})
stds1  = stds1.rename(columns={SOIL_PH_COL:"pH", SOIL_EC_COL:"EC", SOIL_BD_COL:"Bulk Density"})

plot_grouped_bars(means1, stds1,
                  "Figure 1 — Bulk Soil Chemical Parameters (2016)",
                  "Value (use correct units in your synthesis)",
                  OUTDIR / "Figure1_SoilChem_BulkParams.png")

print("Saved Figure 1")


In [ ]:

# Figure 2
means2 = soil2016.groupby(SOIL_SITE_COL)[[SOIL_OM_COL, SOIL_TC_COL]].mean()
stds2  = soil2016.groupby(SOIL_SITE_COL)[[SOIL_OM_COL, SOIL_TC_COL]].std()

means2 = means2.rename(columns={SOIL_OM_COL:"Organic Matter", SOIL_TC_COL:"Total Carbon"})
stds2  = stds2.rename(columns={SOIL_OM_COL:"Organic Matter", SOIL_TC_COL:"Total Carbon"})

plot_grouped_bars(means2, stds2,
                  "Figure 2 — Soil Carbon Characteristics (2016)",
                  "Value (use correct units in your synthesis)",
                  OUTDIR / "Figure2_SoilChem_Carbon.png")

print("Saved Figure 2")


## STEP 7 — Table 2 (Soil temperature: yearly averages)

**Deliverables**
- `outputs/Table2_SoilTemp_YearlyAvg.csv`
- `outputs/Table2_SoilTemp_YearlyAvg.png` (first 25 rows for readability)


In [ ]:

temp2 = temp.copy()
temp2["Date"] = pd.to_datetime(temp2[TEMP_DATE_COL], errors="coerce")
temp2["Year"] = temp2["Date"].dt.year
temp2["TempValue"] = to_numeric_clean(temp2[TEMP_VALUE_COL])

table2 = (
    temp2.dropna(subset=["Year","TempValue"])
        .groupby([TEMP_SITE_COL, "Year"])["TempValue"]
        .mean()
        .reset_index()
        .rename(columns={"TempValue":"MeanSoilTemp"})
)

table2.head()


In [ ]:

table2_csv = OUTDIR / "Table2_SoilTemp_YearlyAvg.csv"
table2_png = OUTDIR / "Table2_SoilTemp_YearlyAvg.png"
table2.to_csv(table2_csv, index=False)
save_df_as_png(table2.head(25), table2_png, title="Table 2 — Yearly Avg Soil Temp (first 25 rows)")
print("Saved:", table2_csv.name, "and", table2_png.name)


## STEP 8 — Figure 3 (Daily soil temperature for plot **E22**)

To keep the plot readable, choose **one year**.

**Deliverable**
- `outputs/Figure3_SoilTemp_E22.png`


In [ ]:

# TODO: choose a year to plot (2016 recommended)
PLOT_YEAR = 2016

temp_e22 = temp2[temp2[TEMP_PLOT_COL].astype(str).str.contains("E22", na=False)].copy()
temp_e22 = temp_e22[temp_e22["Year"] == PLOT_YEAR].copy()

print("Rows in E22 subset for", PLOT_YEAR, ":", len(temp_e22))
temp_e22[[TEMP_SITE_COL, "Date", "TempValue"]].head()


In [ ]:

plt.figure(figsize=(10, 5))
for site, df_site in temp_e22.dropna(subset=["Date","TempValue"]).groupby(TEMP_SITE_COL):
    df_site = df_site.sort_values("Date")
    plt.plot(df_site["Date"], df_site["TempValue"], label=str(site), linewidth=1)

plt.title(f"Figure 3 — Daily Soil Temperature (E22) in {PLOT_YEAR}")
plt.xlabel("Date")
plt.ylabel("Soil temperature (units depend on dataset)")
plt.legend(title="Site", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
save_fig(OUTDIR / "Figure3_SoilTemp_E22.png")
plt.close()
print("Saved Figure 3")


## STEP 9 — Water chemistry cleanup (convert selected columns to numeric)

Excel cleaning should remove most `-` and `<`, but we still convert defensively.


In [ ]:

water2 = water.copy()

# Convert chemistry columns to numeric
for col in WATER_CHEM_COLS:
    water2[col] = to_numeric_clean(water2[col])

# Discharge (optional)
if WATER_DISCHARGE_COL is not None:
    water2[WATER_DISCHARGE_COL] = to_numeric_clean(water2[WATER_DISCHARGE_COL])

# Date (optional)
if WATER_DATE_COL is not None:
    water2["Date"] = pd.to_datetime(water2[WATER_DATE_COL], errors="coerce")

print("Ready for Table 3 / Figure 4.")


## STEP 10 — Table 3 (Bechler River summary stats)

**Deliverables**
- `outputs/Table3_BechlerRiver_Stats.csv`
- `outputs/Table3_BechlerRiver_Stats.png`


In [ ]:

bechler = water2[water2[WATER_SITE_COL].astype(str).str.contains("Bechler", na=False)].copy()
print("Bechler rows:", len(bechler))

cols_for_stats = list(WATER_CHEM_COLS)
if WATER_DISCHARGE_COL is not None:
    cols_for_stats.append(WATER_DISCHARGE_COL)

table3 = (
    bechler[cols_for_stats]
    .agg(["count", "mean", "std", "min", "max"])
    .T
    .reset_index()
    .rename(columns={"index":"Variable"})
)

table3


In [ ]:

table3_csv = OUTDIR / "Table3_BechlerRiver_Stats.csv"
table3_png = OUTDIR / "Table3_BechlerRiver_Stats.png"
table3.to_csv(table3_csv, index=False)
save_df_as_png(table3, table3_png, title="Table 3 — Bechler River Summary Stats")
print("Saved:", table3_csv.name, "and", table3_png.name)


## STEP 11 — Figure 4 (choose a **new** figure type)

Pick ONE figure type not used in Figures 1–3.

**Option A (recommended):** boxplots of chemistry variables.
**Option B:** discharge vs analyte scatter (only if discharge exists).

**Deliverable**
- `outputs/Figure4_WaterChem_Bechler.png`


In [ ]:

# Option A — Boxplots (recommended starter)
data = bechler[WATER_CHEM_COLS].dropna(how="all")

plt.figure(figsize=(10, 5))
plt.boxplot([data[c].dropna().values for c in WATER_CHEM_COLS],
            labels=WATER_CHEM_COLS, vert=True)
plt.xticks(rotation=30, ha="right")
plt.title("Figure 4 — Water Chemistry Distributions (Bechler River)")
plt.ylabel("Concentration (units depend on dataset)")
plt.tight_layout()
save_fig(OUTDIR / "Figure4_WaterChem_Bechler.png")
plt.close()

print("Saved Figure 4 (Option A)")


In [ ]:

# Option B — Scatter (uncomment to use instead of Option A)
# if WATER_DISCHARGE_COL is None:
#     raise ValueError("Set WATER_DISCHARGE_COL to use Option B.")

# ANALYTE_COL = WATER_CHEM_COLS[0]  # TODO pick one
# df = bechler[[WATER_DISCHARGE_COL, ANALYTE_COL]].dropna()

# plt.figure(figsize=(7, 5))
# plt.scatter(df[WATER_DISCHARGE_COL], df[ANALYTE_COL])
# plt.title(f"Figure 4 — {ANALYTE_COL} vs Discharge (Bechler River)")
# plt.xlabel("Discharge (units depend on dataset)")
# plt.ylabel(f"{ANALYTE_COL} (units depend on dataset)")
# plt.tight_layout()
# save_fig(OUTDIR / "Figure4_WaterChem_Bechler.png")
# plt.close()

# print("Saved Figure 4 (Option B)")


## STEP 12 (Optional) — Export GIS-ready CSV for ArcGIS Pro

If your cleaned sheet includes **Latitude/Longitude**, export a CSV for mapping.
If not, skip this step and follow your instructor’s ArcGIS workflow.

**Deliverable (optional)**
- `outputs/Map1_points_for_arcgis.csv`


In [ ]:

# TODO: set these if you have coordinates in ONE sheet (water is most common)
LAT_COL = None   # e.g., "Latitude"
LON_COL = None   # e.g., "Longitude"
NAME_COL = WATER_SITE_COL

if LAT_COL is not None and LON_COL is not None:
    pts = (
        water2[[NAME_COL, LAT_COL, LON_COL]]
        .dropna()
        .drop_duplicates()
        .rename(columns={NAME_COL:"Name", LAT_COL:"Latitude", LON_COL:"Longitude"})
    )
    out_csv = OUTDIR / "Map1_points_for_arcgis.csv"
    pts.to_csv(out_csv, index=False)
    print("Saved:", out_csv.name, "rows:", len(pts))
else:
    print("Skipping GIS export (LAT_COL/LON_COL not set).")


## STEP 13 — Zip your outputs for submission

This creates `outputs.zip` and downloads it.


In [ ]:

zip_path = Path("outputs.zip")
if zip_path.exists():
    zip_path.unlink()

with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for fp in OUTDIR.glob("*"):
        z.write(fp, arcname=f"outputs/{fp.name}")

print("Created:", zip_path, "with", len(list(OUTDIR.glob('*'))), "files")
files.download(str(zip_path))
